In [1]:
from tensorflow.keras.layers import Dense, LSTM, Conv2D, TimeDistributed, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np

In [2]:
frames = 320
x = 480
y = 640
classes = ["Reading", "Drink", "Eat", "Call Cellphone", "Write", "Use Laptop", "Use Vaccum Cleaner", "Cheer Up", "Sit Still", "Toss Paper", "Play Game", "Lay Down", "Walk", "Stand Up", "Sit Down"]
outputs = len(classes)


In [3]:
model = Sequential()
model.add(TimeDistributed(Conv2D(input_shape=(frames, x, y, 1), filters=4,kernel_size=(3,3),padding="same", activation="relu")))
#model.add(TimeDistributed(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")))
model.add(TimeDistributed(GlobalAveragePooling2D()))
model.add(LSTM(50, input_shape=(frames,1), activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(10)))
#model.add(Dense(1024, activation='relu'))
model.add(Dense(outputs, activation='softmax'))

In [4]:
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.build((4,frames,x,y,1))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (4, 320, 480, 640, 4)    40        
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (4, 320, 4)              0         
 tributed)                                                       
                                                                 
 lstm (LSTM)                 (4, 320, 50)              11000     
                                                                 
 time_distributed_2 (TimeDis  (4, 320, 10)             510       
 tributed)                                                       
                                                                 
 dense_1 (Dense)             (4, 320, 15)              165       
                                                        

In [5]:
import cv2 as cv
import os

for f in os.listdir("MSRDailyAct3D_pack2/Reading"):
    if f.endswith(".avi"):
        continue
    os.remove(os.path.join("MSRDailyAct3D_pack2/Reading", f))

i=0
c=0
X_train = []
Y_train = []
for path in os.listdir("MSRDailyAct3D_pack2/Reading"):
    capture = cv.VideoCapture("MSRDailyAct3D_pack2/Reading/"+path)

    #work

    i = i+1   
    
    temp_frames = []
    while(True):
        
        ret, frame = capture.read()
        #print(ret)
        if ret == True:

            grayImage = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            cv.imshow('Frame '+ str(i), grayImage)
            temp_frames.append(grayImage)


        # Break the loop
        else: 
            break
        
    for t in range(frames-len(temp_frames)):
        temp_frames.append(np.zeros((x,y)))
    X = np.array(temp_frames)
    X=X.reshape(X.shape[0],X.shape[1],X.shape[2],1)
    X=X.astype("float16")
    ind = 0
    label = np.zeros((X.shape[0], len(classes)))
    label[:][ind] = 1
    Y_train.append(label)
    #print(label.shape)
    X_train.append(X)

                         
capture.release()
cv.destroyAllWindows()

In [6]:
X_train_final = np.array(X_train)
Y_train_final = np.array(Y_train)

#model.fit(X_train_final, Y_train_final)

In [7]:
for t in range(2):
    print(X_train_final[t].shape)

(320, 480, 640, 1)
(320, 480, 640, 1)


In [ ]:
model.fit(X_train_final, Y_train_final, epochs=5)

Epoch 1/5
